# Determine derivatives of angular sequence to rotation matrix

SymPy code to evaluate the mappings from rotation angle sequences to rotation matrix, then compute the derivatives to determine the mapping from angular rates to angular velocity.

Adjust the next cell to choose the particular angle sequence.

In [225]:
angle_names = ('alpha', 'beta', 'gamma')
func = eul2r

#angle_names =  ('phi', 'theta', 'psi')
#func = lambda Gamma: rpy2r(Gamma, order='xyz')

In [226]:
from spatialmath.base import *
from sympy import *

Define a symbol for time

In [227]:
t = symbols('t')

Define arrays of symbols for the angles, angle as function of time, time derivative of angle as a function of time

In [228]:
angle = []  # names of angles, eg. theta
anglet = []  # angles as function of time, eg. theta(t)
angled = []  # derivative of above, eg. d theta(t) / dt
angledn = []  # symbol to represent above, eg. theta_dot
for i in angle_names:
    angle.append(symbols(i))
    anglet.append(Function(i)(t))
    angled.append(anglet[-1].diff(t))
    angledn.append(i + '_dot')

Compute the rotation matrix

In [229]:
R = Matrix(func(anglet))
R

Matrix([
[-sin(alpha(t))*sin(gamma(t)) + cos(alpha(t))*cos(beta(t))*cos(gamma(t)), -sin(alpha(t))*cos(gamma(t)) - sin(gamma(t))*cos(alpha(t))*cos(beta(t)), sin(beta(t))*cos(alpha(t))],
[ sin(alpha(t))*cos(beta(t))*cos(gamma(t)) + sin(gamma(t))*cos(alpha(t)), -sin(alpha(t))*sin(gamma(t))*cos(beta(t)) + cos(alpha(t))*cos(gamma(t)), sin(alpha(t))*sin(beta(t))],
[                                            -sin(beta(t))*cos(gamma(t)),                                              sin(beta(t))*sin(gamma(t)),               cos(beta(t))]])

Compute its time derivative

In [230]:
Rdot = Matrix(R).diff(t)
Rdot

Matrix([
[-sin(alpha(t))*cos(beta(t))*cos(gamma(t))*Derivative(alpha(t), t) - sin(alpha(t))*cos(gamma(t))*Derivative(gamma(t), t) - sin(beta(t))*cos(alpha(t))*cos(gamma(t))*Derivative(beta(t), t) - sin(gamma(t))*cos(alpha(t))*cos(beta(t))*Derivative(gamma(t), t) - sin(gamma(t))*cos(alpha(t))*Derivative(alpha(t), t), sin(alpha(t))*sin(gamma(t))*cos(beta(t))*Derivative(alpha(t), t) + sin(alpha(t))*sin(gamma(t))*Derivative(gamma(t), t) + sin(beta(t))*sin(gamma(t))*cos(alpha(t))*Derivative(beta(t), t) - cos(alpha(t))*cos(beta(t))*cos(gamma(t))*Derivative(gamma(t), t) - cos(alpha(t))*cos(gamma(t))*Derivative(alpha(t), t), -sin(alpha(t))*sin(beta(t))*Derivative(alpha(t), t) + cos(alpha(t))*cos(beta(t))*Derivative(beta(t), t)],
[-sin(alpha(t))*sin(beta(t))*cos(gamma(t))*Derivative(beta(t), t) - sin(alpha(t))*sin(gamma(t))*cos(beta(t))*Derivative(gamma(t), t) - sin(alpha(t))*sin(gamma(t))*Derivative(alpha(t), t) + cos(alpha(t))*cos(beta(t))*cos(gamma(t))*Derivative(alpha(t), t) + cos(alpha(t))

Get angular velocity vector in terms of angles and angle rates

In [231]:
omega = Matrix(vex(Rdot * R.T))

For each element of this 3x1 matrix get the coefficients of each angle derivative

In [232]:
A = Matrix.zeros(3,3)
for i in range(3):
    e = omega[i,0].expand()
    for j in range(3):
        A[i, j] = e.coeff(angled[j])

The result is a 3x3 matrix. Mapping from angle rates to angular velocity. We subsitute angle as a function of time to plain angle, then simplify.

In [233]:
A = trigsimp(A.subs(a for a in zip(anglet, angle)))

Compute the inverse and simplify.  Mapping from angular velocity to angle rates.

In [234]:
Ai = trigsimp(A.inv())

Render as code

In [235]:
pycode(A).replace('ImmutableDenseMatrix', 'np.array')

'np.array([[0, -math.sin(alpha), math.sin(beta)*math.cos(alpha)], [0, math.cos(alpha), math.sin(alpha)*math.sin(beta)], [1, 0, math.cos(beta)]])'

In [236]:
pycode(Ai).replace('ImmutableDenseMatrix', 'np.array')

'np.array([[-math.cos(alpha)/math.tan(beta), -math.sin(alpha)/math.tan(beta), 1], [-math.sin(alpha), math.cos(alpha), 0], [math.cos(alpha)/math.sin(beta), math.sin(alpha)/math.sin(beta), 0]])'

Compute the time derivative of `Ai`, from angular acceleration to angle acceleration

In [237]:
Ai = Ai.subs(a for a in zip(angle, anglet))

In [238]:
Ai_dot = trigsimp(Ai.diff(t).subs(a for a in zip(angled, angledn)).subs(a for a in zip(anglet, angle)))
Ai_dot

Matrix([
[          alpha_dot*sin(alpha)/tan(beta) + beta_dot*cos(alpha)/sin(beta)**2,         -alpha_dot*cos(alpha)/tan(beta) + beta_dot*sin(alpha)/sin(beta)**2, 0],
[                                                      -alpha_dot*cos(alpha),                                                      -alpha_dot*sin(alpha), 0],
[-(alpha_dot*sin(alpha) + beta_dot*cos(alpha)*cos(beta)/sin(beta))/sin(beta), (alpha_dot*cos(alpha) - beta_dot*sin(alpha)*cos(beta)/sin(beta))/sin(beta), 0]])

In [239]:
pycode(Ai_dot).replace('ImmutableDenseMatrix', 'np.array')

'np.array([[alpha_dot*math.sin(alpha)/math.tan(beta) + beta_dot*math.cos(alpha)/math.sin(beta)**2, -alpha_dot*math.cos(alpha)/math.tan(beta) + beta_dot*math.sin(alpha)/math.sin(beta)**2, 0], [-alpha_dot*math.cos(alpha), -alpha_dot*math.sin(alpha), 0], [-(alpha_dot*math.sin(alpha) + beta_dot*math.cos(alpha)*math.cos(beta)/math.sin(beta))/math.sin(beta), (alpha_dot*math.cos(alpha) - beta_dot*math.sin(alpha)*math.cos(beta)/math.sin(beta))/math.sin(beta), 0]])'